In [1]:
#importing all the required packages
import optuna
from optuna.samplers import GPSampler, RandomSampler

import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools
import os
from pathlib import Path

import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/')
from model.GRU_model import *
from pipeline_config import *
from utils import *

/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs : {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.1
Epochs : 100


In [3]:
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [4]:
def objective(trial):
    with open(train_dataset_path, 'rb') as f:
        train_dataset_dict = pickle.load(f)
        
    with open(test_dataset_path, 'rb') as f:
        test_dataset_dict = pickle.load(f)

    with open(category_id_to_category_name_path, 'rb') as f:
        category_id_to_name_dict = pickle.load(f)
        
    with open(categories_per_indent_path, 'rb') as f:
        categories_per_indent_dict = pickle.load(f)

    with open(son_parent_path, 'rb') as f:
        son_parent_dict = pickle.load(f)
    
    weights_path = weightspath
    #--------------------------------------------------------------------------------------------------------------------------------------#
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights/'

    is_empty = not any(Path(desired_path).iterdir())
    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)

    #--------------------------------------------------------------------------------------------------------------------------------------#

    loss_coef = trial.suggest_float('loss_coef',  1e-10, 1e-1, log=True)
    Lr = trial.suggest_float('Lr', 1e-5, 1e-1, log=True)
    seed = trial.suggest_int('seed', 0 , 100)
    hgru_models = {}

    num_categories = 0
    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]:
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                parent_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=seed)
                parent_optimizer = torch.optim.AdamW(Model.parameters(), lr=Lr)
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(weights_path+parent_name+'.pt', parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model=GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=seed)
            optimizer = torch.optim.AdamW(Model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            hgru_models[category_name] = training_and_evaluation(trial, model, train_dataloader, test_dataloader, optimizer, category, parent_weights, loss_coef, path= saving_param_path)
                

    average_error = sum(list(hgru_models.values()))/len(list(hgru_models.values()))
    
    return average_error


In [5]:
study = optuna.create_study(direction="minimize", sampler=RandomSampler())
study.optimize(objective, n_trials=50)

[I 2024-07-11 09:54:02,249] A new study created in memory with name: no-name-bb646634-5eb5-4307-9ff0-b5b976bd432d


Successfully deleted the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/hgru/models_weights/
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/hgru/models_weights/
num categories: 1
category id|name: 0|All-items
num categories: 2
category id|name: 2|Alcoholic beverages and tobacco


/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python

num categories: 3
category id|name: 3|Clothing and footwear
num categories: 4
category id|name: 8|Communications
num categories: 5
category id|name: 10|Education
num categories: 6
category id|name: 1|Food and non-alcoholic beverages
num categories: 7
category id|name: 5|Furnishings, household equipment and routine maintenance
num categories: 8
category id|name: 6|Health
num categories: 9
category id|name: 4|Housing, water, electricity, gas and other fuels
num categories: 10
category id|name: 12|Miscellaneous goods and services
num categories: 11
category id|name: 9|Recreation and culture
num categories: 12
category id|name: 11|Restaurants and hotels
num categories: 13
category id|name: 7|Transport
num categories: 14
category id|name: 45|Accommodation services
num categories: 15
category id|name: 19|Actual rentals for housing
num categories: 16
category id|name: 15|Alcoholic beverages
num categories: 17
category id|name: 38|Audio-visual, photographic and information processing equipment

[I 2024-07-11 09:59:47,740] Trial 0 finished with value: 0.19596955966035534 and parameters: {'loss_coef': 1.834172587084012e-07, 'Lr': 0.00011913769643199228, 'seed': 65}. Best is trial 0 with value: 0.19596955966035534.


...

[I 2024-07-11 13:05:36,293] Trial 48 finished with value: 0.1945405482406636 and parameters: {'loss_coef': 0.05703578877083479, 'Lr': 0.002569290796403794, 'seed': 39}. Best is trial 13 with value: 0.1925394513956424.


Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/hgru/models_weights/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/hgru/models_weights/
num categories: 1
category id|name: 0|All-items
num categories: 2
category id|name: 2|Alcoholic beverages and tobacco


/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python

num categories: 3
category id|name: 3|Clothing and footwear
num categories: 4
category id|name: 8|Communications
num categories: 5
category id|name: 10|Education
num categories: 6
category id|name: 1|Food and non-alcoholic beverages
num categories: 7
category id|name: 5|Furnishings, household equipment and routine maintenance
num categories: 8
category id|name: 6|Health
num categories: 9
category id|name: 4|Housing, water, electricity, gas and other fuels
num categories: 10
category id|name: 12|Miscellaneous goods and services
num categories: 11
category id|name: 9|Recreation and culture
num categories: 12
category id|name: 11|Restaurants and hotels
num categories: 13
category id|name: 7|Transport
num categories: 14
category id|name: 45|Accommodation services
num categories: 15
category id|name: 19|Actual rentals for housing
num categories: 16
category id|name: 15|Alcoholic beverages
num categories: 17
category id|name: 38|Audio-visual, photographic and information processing equipment

[I 2024-07-11 13:08:26,335] Trial 49 finished with value: 0.19552497819813164 and parameters: {'loss_coef': 4.233280913615921e-05, 'Lr': 0.00034579251058114706, 'seed': 30}. Best is trial 13 with value: 0.1925394513956424.


In [6]:
# Get the best trial
best_trial = study.best_trial

# Print the best trial
print('Best trial:')
print(f'Value: {best_trial.value}')
print('Parameters:')
for key, value in best_trial.params.items():
    print(f'{key}: {value}')

Best trial:
Value: 0.1925394513956424
Parameters:
loss_coef: 5.336535902170534e-07
Lr: 0.00019837551214953915
seed: 0
